In [3]:
!uv pip install "semantic-kernel[google]==1.30.0"

Resolved 113 packages in 2.45s                                       
   Building google-api-python-client==2.171.0                          
   Building google-api-python-client==2.171.0                  
⠙ Preparing packages... (0/28)
   Building google-api-python-client==2.171.0    0 B/79.35 KiB         
⠙ Preparing packages... (0/28)
   Building google-api-python-client==2.171.014.87 KiB/79.35 KiB       
⠙ Preparing packages... (0/28)
   Building google-api-python-client==2.171.030.87 KiB/79.35 KiB       
⠙ Preparing packages... (0/28)
   Building google-api-python-client==2.171.030.87 KiB/79.35 KiB       
⠙ Preparing packages... (0/28)
google-resumable-media ------------------------------ 30.87 KiB/79.35 KiB
   Building google-api-python-client==2.171.0    0 B/108.52 KiB        
⠙ Preparing packages... (0/28)
google-resumable-media ------------------------------ 30.87 KiB/79.35 KiB
   Building google-api-python-client==2.171.014.88 KiB/108.52 KiB      
⠙ Preparing packages... (0/2

In [2]:
import os
import asyncio
from enum import Enum
# import mermaid as md

from semantic_kernel import Kernel
# from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.kernel_pydantic import KernelBaseModel
from semantic_kernel.processes.kernel_process.kernel_process_step import (
    KernelProcessStep,
)
from semantic_kernel.processes.kernel_process.kernel_process_step_context import (
    KernelProcessStepContext,
)
from semantic_kernel.processes.kernel_process.kernel_process_step_state import (
    KernelProcessStepState,
)
from semantic_kernel.processes.local_runtime.local_event import KernelProcessEvent
from semantic_kernel.processes.local_runtime.local_kernel_process import start
from semantic_kernel.processes.process_builder import ProcessBuilder

In [1]:
import os
from dotenv import load_dotenv
from semantic_kernel.connectors.ai.google.google_ai import GoogleAIChatCompletion

load_dotenv()

chat_completion_service = GoogleAIChatCompletion()

/Users/abuzarsultan/sk-gemini/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from semantic_kernel import Kernel

# Initialize the kernel
kernel = Kernel()

# Add the chat completion service created above to the kernel
kernel.add_service(chat_completion_service)

In [6]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase

# Retrieve the chat completion service by type
chat_completion_service = kernel.get_service(type=ChatCompletionClientBase)

In [16]:
from semantic_kernel.connectors.ai.google.google_ai import GoogleAIChatPromptExecutionSettings

execution_settings = GoogleAIChatPromptExecutionSettings(
    response_mime_type="application/json",
    response_schema={
        "type": "object",
        "properties": {
            "response": {"type": "string"},
            "key_points": {
                "type": "array",
                "items": {"type": "string"}
            },
            "sentiment": {
                "type": "string",
                "enum": ["positive", "neutral", "negative"]
            }
        },
        "required": ["response", "key_points", "sentiment"]
    }
)

In [17]:
from semantic_kernel.contents.chat_history import ChatHistory

history = ChatHistory()
history.add_user_message("Hello, how are you?")

response = await chat_completion_service.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
)

print(response)


{"key_points": ["User initiated conversation", "User asked about my well-being"], "response": "Hello there! As an AI, I don't experience 'doing' well in the human sense, but I am operational and ready to assist. How can I help you today?", "sentiment": "positive"}


In [3]:
from semantic_kernel.connectors.ai.google.google_ai import GoogleAIChatCompletion

def create_kernel_with_service(service_id="default"):
    """Create a kernel with Google AI service."""
    kernel = Kernel()

    if (os.getenv("GOOGLE_AI_API_KEY")):
        print("Using Google AI service")
        kernel.add_service(
            GoogleAIChatCompletion(service_id=service_id)
        )
    else:
        raise ValueError(
            "No AI service credentials found. Please set up your .env file."
        )

    return kernel


# Create our kernel
kernel = create_kernel_with_service(service_id="process-framework")
print("Kernel created successfully!")


Using Google AI service
Kernel created successfully!


/Users/abuzarsultan/sk-gemini/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
from enum import Enum
# Define events for our content creation process
class ContentEvents(Enum):
    StartProcess = "startProcess"
    TopicReceived = "topicReceived"
    PlanGenerated = "planGenerated"
    PlanReviewed = "planReviewed"
    RevisionNeeded = "revisionNeeded"
    ReportGenerated = "reportGenerated"
    ReportReviewed = "reportReviewed"
    ReportRevisionNeeded = "reportRevisionNeeded"
    ProcessComplete = "processComplete"


# Define state for the content creation process
class GenerateState(KernelBaseModel):
    topic: str = ""
    plan: str = ""
    review: str = ""
    report: str = ""
    report_review: str = ""
    revision_count: int = 0
    report_revision_count: int = 0
    max_revisions: int = 1

    def debug_info(self):
        return f"State: topic='{self.topic}', plan_length={len(self.plan)}, revision_count={self.revision_count}"


# Step to get the topic from the user
class TopicInputStep(KernelProcessStep[GenerateState]):
    async def activate(self, state: KernelProcessStepState[GenerateState]):
        """Activates the step and sets the state."""
        state.state = state.state or GenerateState()
        self.state = state.state
        # print(f"DEBUG - TopicInputStep activated: {self.state.debug_info()}")

    @kernel_function
    async def get_topic(self, context: KernelProcessStepContext):
        print("What topic would you like your report written about?")
        topic = input("TOPIC: ")

        self.state.topic = topic
        # print(f"DEBUG - Setting topic to: '{self.state.topic}'")

        await context.emit_event(
            process_event=ContentEvents.TopicReceived, data=self.state
        )
        # print(f"DEBUG - Emitted TopicReceived with state.topic = '{self.state.topic}'")


# Step to generate content
class PlanGenerationStep(KernelProcessStep[GenerateState]):
    async def activate(self, state: KernelProcessStepState[GenerateState]):
        """Activates the step and sets the state."""
        state.state = state.state or GenerateState()
        self.state = state.state
        # print(f"DEBUG - PlanGenerationStep activated: {self.state.debug_info()}")

    @kernel_function
    async def generate_plan(
        self,
        context: KernelProcessStepContext,
        kernel: Kernel,
        generate_state: GenerateState = None,
    ):
        """Generates content based on the topic."""
        if generate_state:
            # print(
            #     f"DEBUG - Received generate_state parameter with topic: '{generate_state.topic}'"
            # )
            self.state = generate_state

        print(f"Generating plan about: {self.state.topic}")

        # Get chat completion service and generate content
        chat_service = kernel.get_service(service_id="process-framework")
        settings = chat_service.instantiate_prompt_execution_settings(
            service_id="process-framework"
        )

        chat_history = ChatHistory()
        chat_history.add_system_message(
            f"You are a senior accountant specializing in writing reports about {self.state.topic}. Your performance will be judged on the thoroughness and actionability of the Research Plan you generate, and the accuracy, completeness, and impeccable citation of the factual information you extract when executing that plan. Deliver excellence."
        )
        chat_history.add_user_message(
            f"Write a short plan about how you would structure a report about {self.state.topic}. Keep it concise but informative."
        )

        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history, settings=settings
        )

        if response is None:
            raise ValueError(
                "Failed to get a response from the chat completion service."
            )

        self.state.plan = response[0].content

        print("Plan generated successfully!")
        # print(f"DEBUG - Generated Plan length: {len(self.state.plan)}")

        await context.emit_event(
            process_event=ContentEvents.PlanGenerated, data=self.state
        )
        # print(f"DEBUG - Emitted Plan Generated with state: {self.state.debug_info()}")


# Step to review the content
class PlanReviewStep(KernelProcessStep[GenerateState]):
    async def activate(self, state: KernelProcessStepState[GenerateState]):
        """Activates the step and sets the state."""
        state.state = state.state or GenerateState()
        self.state = state.state
        # print(f"DEBUG - PlanReviewStep activated: {self.state.debug_info()}")

    @kernel_function
    async def review_plan(
        self,
        context: KernelProcessStepContext,
        kernel: Kernel,
        generate_state: GenerateState = None,
    ):
        """Reviews the generated content."""
        if generate_state:
            # print(
            #     f"DEBUG - Received generate_state parameter with topic: '{generate_state.topic}'"
            # )
            self.state = generate_state

        print("Reviewing the generated plan...")

        # Get chat completion service and review content
        chat_service = kernel.get_service(service_id="process-framework")
        settings = chat_service.instantiate_prompt_execution_settings(
            service_id="process-framework"
        )

        chat_history = ChatHistory()
        chat_history.add_system_message(
            "You are a report plan reviewer. Your job is to review the plan for the report and provide feedback."
        )
        chat_history.add_user_message(
            f"Review this plan about {self.state.topic}:\n\n{self.state.plan}\n\nProvide a brief review and state whether it needs revision or is good to publish. If the plan is good to publish, DO NOT include the word 'revision' in your response."
        )

        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history, settings=settings
        )

        if response is None:
            raise ValueError(
                "Failed to get a response from the chat completion service."
            )

        self.state.review = response[0].content

        # print(f"Review: {self.state.review}")

        # Check if revision is needed
        needs_revision = (
            "revision" in self.state.review.lower()
            and self.state.revision_count < self.state.max_revisions
        )

        if needs_revision:
            self.state.revision_count += 1
            # print(
            #     f"Revision needed. Revision count: {self.state.revision_count}/{self.state.max_revisions}"
            # )
            await context.emit_event(
                process_event=ContentEvents.RevisionNeeded, data=self.state
            )
            # print(
            #     f"DEBUG - Emitted RevisionNeeded with state: {self.state.debug_info()}"
            # )
        else:
            print("Content approved or max revisions reached!")
            await context.emit_event(
                process_event=ContentEvents.PlanReviewed, data=self.state
            )
            # print(
            #     f"DEBUG - Emitted PlanReviewed with state: {self.state.debug_info()}"
            # )


# Step to generate the report
class ReportGenerationStep(KernelProcessStep[GenerateState]):
    async def activate(self, state: KernelProcessStepState[GenerateState]):
        state.state = state.state or GenerateState()
        self.state = state.state
        # print(f"DEBUG - ReportGenerationStep activated: {self.state.debug_info()}")

    @kernel_function
    async def generate_report(
        self,
        context: KernelProcessStepContext,
        kernel: Kernel,
        generate_state: GenerateState = None,
    ):
        """Generates the report based on the plan."""
        if generate_state:
            # print(
            #     f"DEBUG - Received generate_state parameter with plan: '{generate_state.plan}'"
            # )
            self.state = generate_state

        print(f"Generating report about: {self.state.topic}")

        # Get chat completion service and generate content
        chat_service = kernel.get_service(service_id="process-framework")
        settings = chat_service.instantiate_prompt_execution_settings(
            service_id="process-framework"
        )

        chat_history = ChatHistory()
        chat_history.add_system_message(
            f"You are a senior accountant specializing in writing reports. Follow the plan and write a detailed report: {self.state.plan}."
        )
        chat_history.add_user_message(
            f"Write a report following the details laid out within the plan: {self.state.plan}."
        )

        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history, settings=settings
        )

        if response is None:
            raise ValueError(
                "Failed to get a response from the chat completion service."
            )

        self.state.report = response[0].content

        print("Report generated successfully!")
        # print(f"Report: {self.state.report}")
        # print(f"DEBUG - Generated Report length: {len(self.state.report)}")

        await context.emit_event(
            process_event=ContentEvents.ReportGenerated, data=self.state
        )
        # print(f"DEBUG - Emitted ReportGenerated with state: {self.state.debug_info()}")


# Step to review the report
class ReportReviewStep(KernelProcessStep[GenerateState]):
    async def activate(self, state: KernelProcessStepState[GenerateState]):
        state.state = state.state or GenerateState()
        self.state = state.state
        # print(f"DEBUG - ReportReviewStep activated: {self.state.debug_info()}")

    @kernel_function
    async def review_report(
        self,
        context: KernelProcessStepContext,
        kernel: Kernel,
        generate_state: GenerateState = None,
    ):
        """Reviews the generated report."""
        if generate_state:
            # print(
            #     f"DEBUG - Received generate_state parameter with topic: '{generate_state.topic}'"
            # )
            self.state = generate_state

        print("Reviewing the generated report...")

        # Get chat completion service and review content
        chat_service = kernel.get_service(service_id="process-framework")
        settings = chat_service.instantiate_prompt_execution_settings(
            service_id="process-framework"
        )

        chat_history = ChatHistory()
        chat_history.add_system_message(
            "You are a report reviewer. Your job is to review the report for the topic and provide feedback. "
        )
        chat_history.add_user_message(
            f"Review this report following the details laid out within the plan: {self.state.plan}:\n\n{self.state.report}\n\nProvide a brief review and state whether it needs revision or is good to publish. If the report is good to publish, DO NOT include the word 'revision' in your response."
        )

        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history, settings=settings
        )

        if response is None:
            raise ValueError(
                "Failed to get a response from the chat completion service."
            )

        self.state.report_review = response[0].content

        #print(f"Review: {self.state.report_review}")

        # Check if revision is needed
        needs_revision = (
            "revision" in self.state.report_review.lower()
            and self.state.report_revision_count < self.state.max_revisions
        )

        if needs_revision:
            self.state.report_revision_count += 1
            #print(
            #    f"Revision needed. Revision count: {self.state.report_revision_count}/{self.state.max_revisions}"
            #)
            await context.emit_event(
                process_event=ContentEvents.ReportRevisionNeeded, data=self.state
            )
            #print(
            #    f"DEBUG - Emitted ReportRevisionNeeded with state: {self.state.debug_info()}"
            # )
        else:
            print("Report approved or max revisions reached!")
            await context.emit_event(
                process_event=ContentEvents.ReportReviewed, data=self.state
            )
            # print(
            #     f"DEBUG - Emitted ReportReviewed with state: {self.state.debug_info()}"
            # )


# Step to finalize the process
class FinalizeStep(KernelProcessStep[GenerateState]):
    async def activate(self, state: KernelProcessStepState[GenerateState]):
        state.state = state.state or GenerateState()
        self.state = state.state
        #print(f"DEBUG - FinalizeStep activated: {self.state.debug_info()}")

    @kernel_function
    async def finalize(
        self, context: KernelProcessStepContext, generate_state: GenerateState = None
    ):
        """Finalizes the process and displays the results."""
        if generate_state:
            # print(
            #     f"DEBUG - Received generate_state parameter with topic: '{generate_state.topic}'"
            # )
            self.state = generate_state

        print("\n" + "=" * 50)
        print("REPORT CREATION PROCESS COMPLETE")
        print("=" * 50)
        print(f"Topic: {self.state.topic}")
        print("-" * 50)
        print(f"Final Plan | plan_length={len(self.state.plan)}:")
        print(self.state.plan)
        print("-" * 50)
        print("Review of the plan:")
        print(self.state.review)
        print("-" * 50)
        print(f"Final Report | report_length={len(self.state.report)}:")
        print(self.state.report)
        print("-" * 50)
        print("Review of the report:")
        print(self.state.report_review)
        print("-" * 50)
        print(f"Plan Revisions: {self.state.revision_count}")
        print("=" * 50 + "\n")
        print(f"Report Revisions: {self.state.report_revision_count}")
        print("=" * 50 + "\n")

        await context.emit_event(process_event=ContentEvents.ProcessComplete, data=None)

In [29]:
# Function to run the report creation process
async def run_report_creation_process():
    # Create our kernel
    kernel = create_kernel_with_service(service_id="process-framework")
    print("Kernel created successfully!")

    # Create a process builder
    process = ProcessBuilder(name="PlanGenerator")

    # Define the steps
    topic_step = process.add_step(TopicInputStep)
    generation_step = process.add_step(PlanGenerationStep)
    review_step = process.add_step(PlanReviewStep)
    report_generation_step = process.add_step(ReportGenerationStep)
    report_review_step = process.add_step(ReportReviewStep)
    finalize_step = process.add_step(FinalizeStep)

    # Define the input event that starts the process
    process.on_input_event(event_id=ContentEvents.StartProcess).send_event_to(
        target=topic_step
    )

    # Define the event flow with explicit parameter mapping
    topic_step.on_event(event_id=ContentEvents.TopicReceived).send_event_to(
        target=generation_step, parameter_name="generate_state"
    )
    generation_step.on_event(event_id=ContentEvents.PlanGenerated).send_event_to(
        target=review_step, parameter_name="generate_state"
    )
    review_step.on_event(event_id=ContentEvents.RevisionNeeded).send_event_to(
        target=generation_step, parameter_name="generate_state"
    )
    review_step.on_event(event_id=ContentEvents.PlanReviewed).send_event_to(
        target=report_generation_step, parameter_name="generate_state"
    )
    report_generation_step.on_event(event_id=ContentEvents.ReportGenerated).send_event_to(
        target=report_review_step, parameter_name="generate_state"
    )
    report_review_step.on_event(event_id=ContentEvents.ReportRevisionNeeded).send_event_to(
        target=report_generation_step, parameter_name="generate_state"
    )
    report_review_step.on_event(event_id=ContentEvents.ReportReviewed).send_event_to(
        target=finalize_step, parameter_name="generate_state"
    )
    finalize_step.on_event(event_id=ContentEvents.ProcessComplete).stop_process()

    # Build the kernel process
    kernel_process = process.build()

    # Start the process
    await start(
        process=kernel_process,
        kernel=kernel,
        initial_event=KernelProcessEvent(id=ContentEvents.StartProcess, data=None),
    )


In [30]:
# Run the report creation process
await run_report_creation_process()

Using Google AI service
Kernel created successfully!
What topic would you like your report written about?
Generating plan about: The evolution of the UK-GAAP and FRS standards in the last 5 years
Plan generated successfully!
Reviewing the generated plan...
Content approved or max revisions reached!
Generating report about: The evolution of the UK-GAAP and FRS standards in the last 5 years
Report generated successfully!
Reviewing the generated report...
Report approved or max revisions reached!

REPORT CREATION PROCESS COMPLETE
Topic: The evolution of the UK-GAAP and FRS standards in the last 5 years
--------------------------------------------------
Final Plan | plan_length=2582:
Okay, here is a concise plan for a report on the evolution of UK-GAAP and FRS standards over the last 5 years:

**Report Plan: Evolution of UK-GAAP and FRS Standards (Last 5 Years)**

1.  **Introduction:**
    *   Define UK-GAAP (Generally Accepted Accounting Practice) and the FRS (Financial Reporting Standard